We will define our genetic algorithm mapper with the following algorithm. For our algorithm, we use the following definition of fitness.
fitness = 1 / (latency  energy)
n = 5, k = 20, p = 10

1. Generate n = 5 randomly ordered strings of the valid dataflow, that is a random permutation of [R, S, P, Q, C, M, N]. Initialize f to 0.
2. Initialize a goal fitness g, dependent on latency and energy.
3. While f>g,
    Mutation: For i from 1 to n mutations, mutate each permutation k/n times to get k mutations. For each mutation, pick two parameters at random and swap them.
    Selection: Calculate latency and energy and evaluate the fitness of each k mutations. Take the p = 10 with the highest fitness.
    Crossover: Take pairs of p = 10 mutations and crossover to get n = 5 permutations. Let f = top fitness from these permutations.
4. Return best permutation.

In [28]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from loaders import *
import yaml
from yaml import load

In [29]:
ARCH_CONFIG = dict( # TODO: need to set this to out config
        DRAM_factor_N=50,
        DRAM_factor_M=8,
        DRAM_factor_C=4,
        global_buffer_factor_N=1,
        global_buffer_factor_M=1,
        global_buffer_factor_C=1,
        PE_spatial_factor_M=1,
        PE_spatial_factor_C=1,
        scratchpad_factor_N=1,
    )

CONFIG = { **ARCH_CONFIG, "pe_meshX": 1, "pe_meshY": 16 } # TODO replace the pe mesh values

In [30]:
def fitness(dataflow, workload):
    # return random.randint(1, 1000)
    data = evaluate(dataflow, workload)
    energy, latency = data
    inverse_EDP = 1 / (energy * latency)
    print(f"{dataflow} has a fitness of {inverse_EDP}")
    return inverse_EDP


def evaluate(dataflow, workload):
    '''
    Evaluates the given dataflow on -- architecture

    dataflow: computation ordering in list format
    workload: the file path to the workload this is being evaluated on
    returns tuple of energy, latency
    '''
    # TODO: should this be done with the constraints file instead?
    # open the map file
    mapping = 'designs/system/map.yaml'
    stream = open(mapping, 'r')
    dictionary = yaml.safe_load(stream)
    print("current mapping: ", dictionary['mapping'][1]['permutation'])
    dictionary['mapping'][1]['permutation'] = dataflow
    print("new mapping: ", dictionary['mapping'][1]['permutation'])
    
    # write the mapping to a new file
    filename = ''.join(dataflow)
    with open(f'iters/configs/{filename}.yaml', 'w') as file:
        yaml.dump(dictionary, file)
    # print(open(f'iters/configs/{filename}.yaml').read())
    
    config = CONFIG
    #TODO write the permutation in designs/system/map.yaml to dataflow
    THRES = (float('inf'), float('inf')) # TODO ?? 
    
    min_energy = float('inf') # TODO: do we need this?
    # for i, k in enumerate(to_run):
    cycle_thres, energy_thres = THRES # TODO whats our threshold
    result = run_timeloop_model( # TODO: this should be run_timeloop_mapper!
        config,
        architecture='designs/system/arch.yaml',
        mapping=mapping,
        problem=workload
    )
    stats = open('./output_dir/timeloop-model.stats.txt', 'r').read()
    mapping = result.mapping
    # print(stats)

    lines = stats.split('\n')
    energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
    cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
    min_energy = min(min_energy, energy)

    print(min_energy, cycles)
    return min_energy, cycles

    

In [31]:
# convolution
dataflow = ['R', 'S', 'P', 'Q', 'C', 'M', 'N']
workload = 'layer_shapes/conv2.yaml'

# n population -> k mutations -> p selection -> n population
# constraints: n | k, p/2 = n
n = 5
k = 20
p = 10

# Generate n base permutations
population = [random.sample(dataflow, len(dataflow)) for i in range(n)]

print("Initializing")
# Initialize base fitness and goal fitness
dfs_fitnesses = [[df, fitness(df, workload)] for df in population]
best_df, f = max(dfs_fitnesses, key=lambda x: x[1])
g = 1000  # if terminating using goal fitness
iter = 10  # if terminating using timeout

for i in range(iter):
    # Mutation
    mutations = []  # len(mutations) = k
    for df in population:
        for m in range(k):
            mutation = df.copy()
            # swap two random indices
            idx1, idx2 = random.sample(range(len(dataflow)), 2)
            mutation[idx1], mutation[idx2] = mutation[idx2], mutation[idx1]
            mutations.append(mutation)
    
    # Selection
    print("selection")
    mutations_fitnesses = [[mutation, fitness(mutation, workload)] for mutation in mutations]
    mutations_fitnesses.sort(key=lambda x: x[1], reverse=True) # high to low fitness
    selections_fitnesses = mutations_fitnesses[:p] 
    selections = [x[0] for x in selections_fitnesses] # len(selections) = p

    # Crossover
    print("crossover")
    random.shuffle(selections)
    crossover_pairs = [(selections[i], selections[i+1]) for i in range(0, len(selections), 2)]
    crossovers = []  # len(crossovers) = n
    for pair in crossover_pairs:
        s1, s2 = pair
        cut_point = random.randint(1, len(s1) - 1)
        first_half = s1[:cut_point]
        second_half = s2.copy()
        for parameter in first_half:
            second_half.remove(parameter)
        crossover = first_half + second_half
        crossovers.append(crossover)

    crossovers_fitnesses = [[crossover, fitness(crossover, workload)] for crossover in crossovers]
    best_df_trial, f_trial = max(crossovers_fitnesses, key=lambda x: x[1])
    if f_trial > f:
        best_df, f = best_df_trial, f_trial
    if f >= g:
        break

Initializing
current mapping:  ['S', 'R', 'Q', 'P', 'C', 'M', 'N']
new mapping:  ['S', 'M', 'N', 'P', 'C', 'Q', 'R']
[INFO] 2025-04-29 20:39:20,707 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


657.11 31360000
['S', 'M', 'N', 'P', 'C', 'Q', 'R'] has a fitness of 4.8527271084051096e-11
current mapping:  ['S', 'R', 'Q', 'P', 'C', 'M', 'N']
new mapping:  ['P', 'R', 'C', 'M', 'N', 'Q', 'S']
[INFO] 2025-04-29 20:39:22,985 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


657.11 31360000
['P', 'R', 'C', 'M', 'N', 'Q', 'S'] has a fitness of 4.8527271084051096e-11
current mapping:  ['S', 'R', 'Q', 'P', 'C', 'M', 'N']
new mapping:  ['P', 'N', 'C', 'M', 'Q', 'S', 'R']
[INFO] 2025-04-29 20:39:25,217 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


...

657.11 31360000
['S', 'P', 'N', 'M', 'C', 'Q', 'R'] has a fitness of 4.8527271084051096e-11
current mapping:  ['S', 'R', 'Q', 'P', 'C', 'M', 'N']
new mapping:  ['S', 'Q', 'N', 'P', 'C', 'M', 'R']
[INFO] 2025-04-29 20:40:10,830 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


657.11 31360000
['S', 'Q', 'N', 'P', 'C', 'M', 'R'] has a fitness of 4.8527271084051096e-11
current mapping:  ['S', 'R', 'Q', 'P', 'C', 'M', 'N']
new mapping:  ['S', 'M', 'C', 'P', 'N', 'Q', 'R']
[INFO] 2025-04-29 20:40:13,141 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


RuntimeError: 

========================================================================================================================
Timeloop model failed with return code -2. Please check the output files in ./output_dir for more information. To debug, you can edit the file:
	./output_dir/parsed-processed-input.yaml
and run 
	tl model ./output_dir/parsed-processed-input.yaml
to see the error. If you're running the mapper and Timeloop can't find a vaild mapping, try setting 'diagnostics: true' in the mapper input specification.